In [6]:
from pprint import pprint
import os
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta


def check_folder_in_path(path, folder_name):
    path_parts = path.split(os.sep)
    return folder_name in path_parts


def merge_csv(folder):
    comments_dfs = []
    posts_dfs = []
    for root, dirs, files in os.walk(os.path.join("./", folder)):
        for file_name in files:
            if not file_name.endswith('.csv'):
                continue
            if not check_folder_in_path(root, "comments"):
                continue
            if not check_folder_in_path(root, "naver_cafe"):
                continue
            file_path = os.path.join(root, file_name)
            df = pd.read_csv(file_path)
            comments_dfs.append(df)
        for file_name in files:
            if not file_name.endswith('.csv'):
                continue
            if not check_folder_in_path(root, "posts"):
                continue
            if not check_folder_in_path(root, "naver_cafe"):
                continue
            file_path = os.path.join(root, file_name)
            df = pd.read_csv(file_path)
            posts_dfs.append(df)

    # 모든 데이터프레임을 하나로 합치기
    comments = pd.concat(comments_dfs, ignore_index=True)
    posts = pd.concat(posts_dfs, ignore_index=True)
    return posts, comments


# 문자열을 정수형으로 변환하는 함수 정의
def convert_to_int(value):
    if type(value) == int:
        return int(value)
    if '만' in value:
        # '만'을 기준으로 변환 (1만 = 10,000)
        return int(float(value.replace('만', '').strip()) * 10000)
    else:
        # 쉼표 제거 후 정수형으로 변환
        return int(value.replace(',', '').strip())


def show(posts, comments):
    # created_at 열을 datetime 형식으로 변환
    posts['created_at'] = pd.to_datetime(posts['created_at'])  # 'created_at'이 실제 날짜 열 이름입니다.
    # 댓글 수 계산을 위해 post_id로 그룹화
    comment_counts = comments.groupby('post_id').size().reset_index(name='comment_count')
    # 댓글 수를 원본 게시물 데이터와 조인
    posts = posts.merge(comment_counts, how='left', left_on='id', right_on='post_id')
    posts['comment_count'] = posts['comment_count'].fillna(0).astype(int)
    # 변환 적용
    posts['views'] = posts['views'].apply(convert_to_int)
    # 각 컬럼의 평균값 계산
    mean_comments = posts['comment_count'].mean()
    mean_views = posts['views'].mean()
    mean_likes = posts['likes'].mean()
    # 가중치를 적용하여 영향력 점수 계산
    posts['influence_score'] = (
            (posts['comment_count'] / mean_comments) +
            (posts['views'] / mean_views) +
            (posts['likes'] / mean_likes)
    )
    # 영향력 점수로 정렬하여 상위 게시물 찾기
    top_influential_posts = posts.sort_values(by='influence_score', ascending=False)
    # 날짜 범위 생성
    date_range = pd.date_range(start=posts['created_at'].min(), end=posts['created_at'].max())
    daily_posts = posts.groupby(posts['created_at'].dt.date).size()
    # 시각화
    fig = go.Figure()
    # 일별 게시글 생성 개수 선 그래프
    fig.add_trace(go.Scatter(x=daily_posts.index, y=daily_posts.values, mode='lines+markers', name='일별 게시글 생성 개수'))
    for date in top_influential_posts['created_at'].head(10):
        fig.add_vline(x=date, line=dict(color="red", width=2))
    # 그래프 설정
    fig.update_layout(title='각 날짜별로 게시물 수',
                      xaxis_title='날짜',
                      yaxis_title='게시글 수',
                      xaxis_tickformat='%Y-%m-%d',
                      legend=dict(x=0, y=1, traceorder='normal', orientation='h'))
    fig.show()
    display(top_influential_posts.head(10).sort_values(by='created_at',ascending=True))

In [22]:
def merge_csv_folder(path):
    """
    path에 있는 모든 csv 파일을 읽어서 하나의 데이터프레임으로 합친다.
    """
    dfs = []
    for root, dirs, files in os.walk(path):
        for file_name in files:
            if not file_name.endswith('.csv'):
                continue
            file_path = os.path.join(root, file_name)
            df = pd.read_csv(file_path)
            dfs.append(df)
        print(len(dfs))
    if len(dfs) == 0:
        return None
    return pd.concat(dfs, ignore_index=True)

# 사이트별로 데이터를 합치기
사이트_목록 = ['bobae', 'clien', 'fmkorea', 'naver_cafe']
사건_목록 = ["벤츠 화재", "아이오닉 누수", "아이오닉 iccu", "코나 화재"]
게시글_댓글 = ["comments", "posts"]

# 수, 사건, 사이트, 종류 를 컬럼으로 가지는 데이터 프레임
df_table = pd.DataFrame(columns=["수", "사건", "사이트", "종류"])


"벤츠 화재/bobae/comments"
"사건/사이트/게시글_댓글"
for 사건 in 사건_목록:
    for 사이트 in 사이트_목록:
        for 종류 in 게시글_댓글:
            df = merge_csv_folder(f"{사건}/{사이트}/{종류}")
            if df is None:
                continue
            # 새로운 데이터를 딕셔너리 형태로 생성
            new_row = {"수": len(df), "사건": 사건, "사이트": 사이트, "종류": 종류}

            # 딕셔너리를 DataFrame으로 변환한 뒤 기존 DataFrame과 병합
            df_table = pd.concat([df_table, pd.DataFrame([new_row])], ignore_index=True)
            
# df_table에서 사건 사이트 별로 수를 합산
df_table_sum = df_table.groupby(["사건", "사이트", "종류"]).sum().reset_index()

# 사이트 순서로 정렬
df_table_sum = df_table_sum.sort_values(by=["사이트", "사건", "종류"], ascending=True)
df_table_sum

# 사건, 사이트 별로 수를 더한 행을 추가
df_table_sum_both = df_table_sum.groupby(["사건", "사이트"]).sum().reset_index()
df_table_sum_both

# df_table_sum에 df_table_sum_both을 사건 사이트 별로 조인, 컬럼명이 겹치도록 함
df_table_joined = df_table_sum.merge(df_table_sum_both, on=["사건", "사이트"])
df_table_joined

# 이걸 사이트 별로 컬럼으로 만들어서 수평적 확장을 해야함

# 사건           bobae, cline, fmkorea, naver_cafe
# 벤츠 화재
# 벤츠 화재 게시글
# 벤츠 화재 댓글
# 아이오닉 누수
# 아이오닉 누수 게시글
# 아이오닉 누수 댓글
# 아이오닉 iccu
# 아이오닉 iccu 게시글
# 아이오닉 iccu 댓글
# 코나 화재
# 코나 화재 게시글
# 코나 화재 댓글



13
13
3
3
17
17
2
2
1
1
2
2
2
2
17
17
7
7
40
40
30
30
1
1


,사건,사이트,종류,수,종류_total,수_total
0,벤츠 화재,bobae,comments,96,commentsposts,208
1,벤츠 화재,bobae,posts,112,commentsposts,208
2,아이오닉 iccu,bobae,comments,12,commentsposts,28
3,아이오닉 iccu,bobae,posts,16,commentsposts,28
4,아이오닉 누수,bobae,comments,11,commentsposts,26
5,아이오닉 누수,bobae,posts,15,commentsposts,26
6,코나 화재,bobae,comments,202,commentsposts,531
7,코나 화재,bobae,posts,329,commentsposts,531
8,벤츠 화재,clien,comments,1545,commentsposts,1603
9,벤츠 화재,clien,posts,58,commentsposts,1603


In [23]:
df = pd.DataFrame(df_table_joined)

# 사건 및 사이트별로 크로스탭 생성
crosstab = pd.pivot_table(df, index=["사건"], columns=["사이트", "종류"], values="수", aggfunc='sum', fill_value=0)

# 행과 열 이름을 적절하게 변경
crosstab.columns = [f"{site} {ctype}" for site, ctype in crosstab.columns]

# 예쁘게 출력
crosstab

/var/folders/v4/6tjs7h4921v9vrm_844m1clw0000gn/T/ipykernel_47895/241822239.py:4: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,bobae comments,bobae posts,clien comments,clien posts,fmkorea comments,fmkorea posts
사건,,,,,,
벤츠 화재,96,112,1545,58,1624,165
아이오닉 iccu,12,16,8176,295,545,61
아이오닉 누수,11,15,375,10,147,18
코나 화재,202,329,13105,514,349,31


In [7]:
사건_목록 = ["벤츠 화재", "아이오닉 누수", "아이오닉 iccu", "코나 화재"]

# # 데이터프레임 생성
# df_statistic = pd.DataFrame(columns=["데이터 종류", "벤츠 화재", "아이오닉 누수", "아이오닉 iccu", "코나 화재"])

# for 사건 in 사건_목록:
#     posts, comments = merge_csv(사건)
    
#     num_posts = len(posts)
#     num_comments = len(comments)
#     데이터_종류 = ["게시글 수", "댓글 수"]
#     show(posts, comments)

posts, comments = merge_csv("코나 화재")
show(posts, comments)

[./코나 화재], [['fmkorea', 'naver_cafe', 'clien', 'bobae']], [[]]
[./코나 화재/fmkorea], [['comments', 'posts']], [[]]
[./코나 화재/fmkorea/comments], [[]], [['1.csv']]
[./코나 화재/fmkorea/posts], [[]], [['1.csv']]
[./코나 화재/naver_cafe], [['2022-07-31', '2020-07-31', '2019-07-31', '2024-07-31', '2021-07-31', '2023-07-31', '2021-01-31', '2023-01-31', '2022-01-31', '2020-01-31', '2024-01-31']], [[]]
[./코나 화재/naver_cafe/2022-07-31], [['comments', 'posts']], [[]]
[./코나 화재/naver_cafe/2022-07-31/comments], [[]], [['1.csv', '3.csv', '2.csv']]
[./코나 화재/naver_cafe/2022-07-31/posts], [[]], [['1.csv', '3.csv', '2.csv']]
[./코나 화재/naver_cafe/2020-07-31], [['comments', 'posts']], [[]]
[./코나 화재/naver_cafe/2020-07-31/comments], [[]], [['6.csv', '7.csv', '5.csv', '4.csv', '1.csv', '3.csv', '2.csv', '23.csv', '22.csv', '20.csv', '21.csv', '25.csv', '18.csv', '30.csv', '24.csv', '26.csv', '27.csv', '16.csv', '17.csv', '29.csv', '15.csv', '14.csv', '28.csv', '10.csv', '11.csv', '13.csv', '12.csv', '9.csv', '8.csv']]
[./

,id,title,content,likes,url,author,views,created_at,updated_at,post_id,comment_count,influence_score
943,505585,현대자동차 코나 EV 배터리 안전성 관련 안내와 화재 관련 전달 사항입니다.,"안녕하세요. 전기차 동호회 운영자입니다.많은 회원들의 국민청원이 진행중이고, 리콜 ...",69.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,매니저,5599,2020-09-11 20:25:00,NaN,505585.0,149,69.461711
712,512987,아파트 전기차 주차장 화재로 대피중입니다.,대구 테크노폴리스 반도유보라 아파트입니다.편히 자고 있는데 대피 안내 방송이 나와 ...,37.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,쏘밍파I아이오닉I대구,11000,2020-10-04 03:28:00,NaN,512987.0,187,52.086910
495,515470,[현대자동차] 코나 EV 화재 리콜 관련 안내사항과 Q&A 전달 사항입니다.,[현대자동차] 코나 EV 화재 리콜 관련 안내사항과 Q&A 전달 사항을 전기차 동호...,38.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,매니저,8270,2020-10-08 18:16:00,NaN,515470.0,157,48.236336
1079,522350,구사일생했습니다,그냥 포기하고 넘어 가려다가 또 다른 피해자가 나올까봐 글을 씁니다2020.10.1...,46.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,Reborn,6762,2020-10-23 18:19:00,NaN,522350.0,155,52.915823
241,554567,2021년 전기차 보조금 정책이 나왔습니다.,1. 국고 보조금2. 지자체 보조금3. 보조금 지급 총액: 국고보조금 + 지자체 보...,59.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,닥침I볼트M3I경기,24000,2021-01-21 14:23:00,NaN,554567.0,361,93.493774
1977,569011,코나 EV 차주로서 허심탄회하게 적습니다...,안녕하세요.코나 EV 차주입니다.저는 2018년에 보조금 소진되서 결국 아주 싼가격...,61.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,행복한부자I코나I서울,4683,2021-02-21 23:45:00,NaN,569011.0,68,56.980263
1482,655521,현대에서 제 차를 안 돌려줍니다.,"7월1일 화재난 코나....7월5일 1차 화재감식 하고, 그 이후에 화재조사 진척이...",52.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,어슬렁 코나 세종,9385,2021-08-05 10:07:00,NaN,655521.0,174,61.444284
1505,761054,코나EV 화재로 바라본 근본적인 문제.,"1. INTRO코나 EV는 단종된 차량입니다. ""국내 시장 수요를 고려한 판매 라인...",56.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,드드드I코나EVI경기,2748,2022-01-12 18:43:00,NaN,761054.0,41,49.388903
1659,1267502,누가 계속 LFP는 고방전이 위험하다고 주장합니까?,실제 BYD에서 생산 중인 LFP 배터리의 자료입니다.BYD Seal/Han/Tan...,32.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,베멤l모델3l부산,4024,2023-08-17 12:34:00,NaN,1267502.0,345,51.651574
2529,1511541,e-GMP의 플라스틱 소재와 열폭주 개선 사항 [1/2],"최근에실내로 화염이 유입된 평택 EV6 화재 사고 관련해서, e-GMP 배터리팩의플...",102.0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,베멤l모델3l부산,9976,2024-07-20 11:52:00,NaN,1511541.0,302,109.691844


In [44]:
posts, comments = merge_csv("아이오닉 누수")
show(posts, comments)

,id,title,content,likes,url,author,views,created_at,updated_at,post_id,comment_count,influence_score
142,627199,현대 전기차 아이오닉5 ‘냉각수 소실’ 문제 발생…자칫 화재로 이어질 가능성,현대 전기차 아이오닉5 ‘냉각수 소실’ 문제 발생…자칫 화재로 이어질 가능성현대자동...,3,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,뭉치I오닉5모3I서울,5082,2021-06-16 11:08:00,NaN,627199.0,39,16.909595
136,637719,EQA250 주행거리 질문있어요!!!!,안녕하세요 9월 달 벤츠 eqa250 받기로 되어잇는 예비 차주입니다.주행거리 42...,0,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,청룡IEQAI의정부,1303,2021-07-05 17:40:00,NaN,637719.0,116,11.803246
8,694014,두대다 운용중 아이오닉과 테슬라 모델3 비교,전기차로는 2018년 중순 넥쏘로 입문해 3만키로 테슬라 모델3퍼포를 2020년 2...,3,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,샤랴샤lm3p아5ㅣ서울,1746,2021-10-04 20:36:00,NaN,694014.0,40,12.497829
22,1319521,"[제보는 MBC] 빗물 유입되는 아이오닉6‥현대차 ""결함 아냐""","[제보는 MBC] 빗물 유입되는 아이오닉6‥현대차 ""결함 아냐""현대자동차의 전기차죠...",4,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,톰I아5iX3모YI서울,2527,2023-10-26 22:16:00,NaN,1319521.0,35,15.344343
93,1320482,아이오닉6 누수문제?,아이오닉6구입하신분들누수문제는 복불복 아니고100% 인가요?ㅜ현대측에서는 결함이아닌...,1,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,콩심이l화성,7300,2023-10-28 08:59:00,NaN,1320482.0,17,13.542268
58,1331941,토레스 안습이네요,앞으로도 현기 독점 하겠네요 ㅜㅜ,4,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,263kmlev6l경북,4186,2023-11-12 22:00:00,NaN,1331941.0,96,22.864328
33,1392740,[팝니다] 아이오닉6 AWD 프레스티지 풀옵션 판매 합니다(선루프X),"[전기차 동호회 직거래 양식]매매가격 : 4,100,000 원1. 차량명과 차량번호...",9,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,환타l서울,1456,2024-02-10 11:23:00,NaN,1392740.0,18,23.559953
208,1474551,대가리 총 맞지 않은 이상 하이랜드!,현기차 아무리 보고 아6타지만 죽어도 현기차 다시 탈 일 없습니다.품질 개 쓰레기에...,5,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,아메l모3예정l경기,2220,2024-06-05 23:30:00,NaN,1474551.0,89,21.833486
184,1482798,전기차는 오너가 되어봐야.....,만족감이 너무 좋습니다.2년 되어가는데. 아직 차에 들어간 소모품비 발생이 아예 없...,5,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,희야I아이오닉6I강원,1351,2024-06-16 00:18:00,NaN,1482798.0,14,14.166285
171,1493399,아이오닉5 비전루프 자랑 좀 해도 될까요,이미 흔하디 흔한 옵션이고 테슬라처럼 통유리는 아니지만 그래도 이런 차가 첨이라 너...,6,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,가비I아이오닉5I청주,1954,2024-06-28 17:42:00,NaN,1493399.0,32,18.764704


In [45]:
posts, comments = merge_csv("벤츠 화재")
show(posts, comments)

,id,title,content,likes,url,author,views,created_at,updated_at,post_id,comment_count,influence_score
362,1524473,청라 아파트 지하서 벤츠 eqe 전기차 화재 폭발 '아수라장'…8시간 동안 불 못껐다,충전도 안 하던 전기차 '펑'…연기 타고 올라 주민 200명 대피청라 아파트 지하서...,3,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,톰I서울,18000,2024-08-01 20:42:00,NaN,1524473.0,55,15.939849
361,1524499,"eqe 화재를 보며,, 타인에 대한 조롱의 화살은 끝내 자기 자신에게 돌아옵니다.",1)서구 청라에서 eqe 차량에서 화재가 발생했다는 뉴스를 봤습니다.cctv영상도 ...,28,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,베니IEQAI인천,2749,2024-08-01 21:15:00,NaN,1524499.0,37,19.410878
611,1526658,지하주차장 화재가 100억 보상보다 더 큰 문제,"전기차 화재로1000도 넘는 발열로 인해기둥골조, 천장골조 전부 녹아내려서아파트 붕...",12,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,땅부I모3I충남,33000,2024-08-03 16:59:00,NaN,1526658.0,40,29.201598
595,1527073,이번화재 의문인점,여태것 주차장화재는 많았는데이렇게크게화재는 처음임,4,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,아이li4l28,19000,2024-08-04 01:14:00,NaN,1527073.0,30,15.615351
576,1527416,청라 화재사고 차주.,일단은 저는 화재사고 본인은 아니구요화재사고 차주분도 분명 이카페에 가입을 하셨을 ...,17,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,팅커IEQE350I인천,6941,2024-08-04 13:33:00,NaN,1527416.0,73,18.018397
671,1530462,1년에 전기차 화재는 얼마나 일어날까?,"최근 큰 화재사건으로 제법 떠들썩한 분위기인데, 이 와중에 기사 한 줄 가져와서 급...",30,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,디오니ㅣ모3ㅣ서울,2126,2024-08-06 20:22:00,NaN,1530462.0,35,20.020011
723,1531278,벤츠 EQE차주입니다 단체행동 부탁드립니다 !!!!,안녕하십니까? 벤츠EQE350+ 차주입니다1. 제 차의 배터리 제조사를 알려주십시요...,39,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,유샘,6536,2024-08-07 12:40:00,NaN,1531278.0,102,31.698327
70,1531857,EQE 차주가 보는 문제점 및 벤츠코리아,안녕하세요~22년12월 와이프 가 탈 용도로 EQE 350 + 할인 딱 300받고 ...,19,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,마산창투IEQEI창원,5374,2024-08-07 19:46:00,NaN,1531857.0,47,16.620316
69,1531865,"전기차로 이슈거리 만들며, 공포감을 형성하는 언론은 이 행위를 당장 멈추어야 합니다.","청라 벤츠 전기차 화재의 본질은 ""전기차"" 가 아니라 스프링클러가 작동하지 않아서 ...",85,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,웅우우웅웅lix3l구리,2308,2024-08-07 19:59:00,NaN,1531865.0,32,50.449854
188,1538590,벤츠 전기차주분들 모두 집중해주십시오!!!!,여기계신 동호회분들의 도움으로 국토부 민원 릴레이중에오늘 벤츠에서 배터리 제조사가 ...,42,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,유샘 EQE350 경기,2388,2024-08-13 17:13:00,NaN,1538590.0,78,29.396909


In [46]:
posts, comments = merge_csv("아이오닉 iccu")
show(posts, comments)

,id,title,content,likes,url,author,views,created_at,updated_at,post_id,comment_count,influence_score
1,762167,아이오닉5 결함/문제점 내용 정리!,안녕하세요.아이오닉5를 21년8월에 받아서 11000km 운행 하였습니다.그동안 결...,19,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,린크I오닉5I경기,3613,2022-01-14 12:00:00,NaN,762167.0,38,27.672873
163,1395665,ICCU 터짐 조사,"안녕하세요, ICCU 에 대한 글이 많이 올라고 있는데, 우리 같이 댓글 조사로 I...",10,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,캔아이오닉5,4812,2024-02-15 11:47:00,NaN,1395665.0,186,28.663298
148,1397954,전기차 구매 신중하셨으면 좋겠네요,"전기차 초보자에 가까운 전린이중의 전린이입니다다만, 이런저런 카페글도 수없이 보고 ...",23,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,펑IEV9l부산,6007,2024-02-19 11:17:00,NaN,1397954.0,68,36.741269
102,1400343,EV9 약 2개월 1000KM 운행후기,"남들은 1주일이면 1000km도 운행한다는데, 1000km 운행하기까지 정말 오래 ...",17,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,펑IEV9l부산,3273,2024-02-22 18:46:00,NaN,1400343.0,67,26.969892
79,1403367,아이오닉5와 토레스evx 소유자로써 느낀점을 정리해 봤습니다.,아이오닉5랑 토레스evx 두차량 소유하고 있습니다. 주로 아이오닉5 제가 몰고 토레...,29,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,태양I아닉5I인천,8636,2024-02-27 10:29:00,NaN,1403367.0,66,46.200141
426,1466119,더뉴아이오닉5 awd 고속 주행 중 멈춤,어제 금요일 저녁에 차 받고 임시번호판으로 운행 중.. 고속도로 주행 중에 터져서긴...,45,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,대구l아5l불도저,7647,2024-05-25 21:30:00,NaN,1466119.0,251,75.103991
534,1467456,수정 뉴아이오닉5 awd 고속 주행 중 멈춤 정리글 [통화녹음o] 마무리,이전 글 :https://cafe.naver.com/allfm01/1466119안녕...,19,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,대구l아5l불도저,2942,2024-05-27 22:28:00,NaN,1467456.0,51,27.786999
565,1476949,토레스EVX 6개월 실차주 후기!!,차가격은 약 5200만원에 보조금 1000만원 조금 덜 받고 선수금 1000만원 넣...,11,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,대전I토레스EVI하윤,3933,2024-06-09 07:43:00,NaN,1476949.0,76,21.592548
677,1507416,전기차 고르는 팁 알려줍니다! 스압,ID4 구매 예정자인데 출고정지 기간이라 시간이 널널하여 그 동안 검색한 내용 정리...,17,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,천재IID4I진주,2547,2024-07-15 20:48:00,NaN,1507416.0,29,23.676682
598,1524499,"eqe 화재를 보며,, 타인에 대한 조롱의 화살은 끝내 자기 자신에게 돌아옵니다.",1)서구 청라에서 eqe 차량에서 화재가 발생했다는 뉴스를 봤습니다.cctv영상도 ...,28,https://cafe.naver.com/ca-fe/ArticleRead.nhn?c...,베니IEQAI인천,2748,2024-08-01 21:15:00,NaN,1524499.0,37,36.696835
